# Datamodels Retriever Experiment

This document has the goal to show the implementation of a context retriever using Datamodels and its comparison against classical approaches

# 1. Run Classical Approaches

First it's needed to run the comparison subject, to do so we will use the script present in this folder, it's just necessary to run

```
python run_classical_retriever.py
```

This will run the retriever for each sample from the test dataset, saving the data at every 50 samples as checkpoint

# 2. Split Data for Datamodeling

Here we will be spliting the data to achieve a dev dataset containing a representative numbe of samples to each subtask
The "k" used here is 15

In [1]:
from src.utils import split_dev_set, subset_df
from src.retriever import DatamodelsRetriever
from src.datamodels import Datamodels, DatamodelConfig
from src.llms import Llama3_1
import pandas as pd

import os

# Limit available GPUs to GPU 0 and 1
os.environ["CUDA_VISIBLE_DEVICES"] = "4"


/home/caio.rhoden/miniconda3/envs/datamodels/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv("../../data/instruction-induction-data/processed/induce_tasks_examples.csv")
# subset_df(train, 200, "task").to_csv("../../data/instruction-induction-data/processed/train.csv")


# split_dev_set(
#     path="../../data/instruction-induction-data/processed/train.csv",
#     saving_path="../../data/instruction-induction-data/datamodels",    
#     k_samples=15,
#     task_column="task",
# )

## 3. Split the Collections to be trained

In [3]:
import pandas as pd
train_set = pd.read_csv("../../data/instruction-induction-data/datamodels/train_set.csv")
train_set

,task,input,output,possible_outputs
0,antonyms,wholly,partly,NaN
1,antonyms,regular,irregular,NaN
2,antonyms,mother,father,NaN
3,antonyms,separate,joint,NaN
4,antonyms,decisive,indecisive,NaN
...,...,...,...,...
3880,word_in_context,Sentence 1: He was hospitalized for extreme fa...,not the same,"['not the same', 'no', 'false']"
3881,word_in_context,Sentence 1: A cartilaginous style. Sentence 2:...,same,"['same', 'yes', 'true']"
3882,word_in_context,Sentence 1: Do you have a light? Sentence 2: T...,not the same,"['not the same', 'no', 'false']"
3883,word_in_context,Sentence 1: Blow on the soup to cool it down. ...,not the same,"['not the same', 'no', 'false']"


In [4]:
##### First time create collection #####
# retriever = DatamodelsRetriever(k=8)
# retriever.create_collections_index(
#     train_set,
#     "../../data/instruction-induction-data/datamodels",
#     n_samples=500,
#     test_per=0.2,

# )

In [5]:
llama = Llama3_1()
llama.run("What is the best vegatable for salad")

Loading checkpoint shards: 100%|██████████| 4/4 [02:31<00:00, 37.96s/it]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/home/caio.rhoden/miniconda3/envs/datamodels/lib/python3.11/site-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input

'?\nI think lettuce is the best vegetable for salad. It has a crisp texture and a fresh taste that makes it a perfect'

In [6]:
######### Datamodels config for experiment #########
config = DatamodelConfig(

    k = 8,
    train_collections_idx_path = "../../data/instruction-induction-data/datamodels/train_collection.h5",
    train_collections_idx = None,
    test_collections_idx_path = "../../data/instruction-induction-data/datamodels/test_collection.h5",
    test_collections_idx = None,
    test_set = None,
    test_set_path = "../../data/instruction-induction-data/datamodels/dev_set.csv",
    train_set = None,
    train_set_path = "../../data/instruction-induction-data/datamodels/train_set.csv",
    collections_path = "../../data/instruction-induction-data/datamodels/collections/",
    llm = llama,
    model =  None,

)

In [7]:
datamodel = Datamodels(config)
datamodel.get_test_set()
datamodel.get_train_set()
datamodel.get_train_collection_index()

Loaded test set from  ../../data/instruction-induction-data/datamodels/dev_set.csv
Loaded train set from  ../../data/instruction-induction-data/datamodels/train_set.csv
Loaded train collection index from  ../../data/instruction-induction-data/datamodels/train_collection.h5


array([[2416, 3636,  410, ..., 3009,  240,  228],
       [2281,  106, 2858, ...,  215, 2256, 3266],
       [2076, 2394, 3168, ..., 2807,  621, 1324],
       ...,
       [2386, 3193, 1778, ..., 2491, 3321, 2297],
       [1311, 1337, 3518, ..., 3391, 3519, 1661],
       [3005, 1634, 1951, ..., 1037, 2231, 1672]])

In [8]:
datamodel.create_collection()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/home/caio.rhoden/miniconda3/envs/datamodels/lib/python3.11/site-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 

 The judge mentioned the manager.

            Input:
            He was a happy person.

            Output:
         He was a happy person


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 1 1 1 1 1 1 1 1 1 1 1 1 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

 same



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 same



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 same

            Input:
            Sentence 1: The man on the street corner was arrested. Sentence 2: The man
 same

